# Resonators with Purcell filters

This is a script to draw an arbitrary number of resonators coupled to a transmission line via another resonator which functions as a Purcell filter.

Requirements: gdspy library.
The design is generated in the following order:


   *   **Ground and contact pads**. THe idea is that you should only specify the coordinates of the contact pads; whether they should be rotated should be decided by the code itself (if the contact pad is further that 900 $\mu m$ away, it's rotated). This is done by the functions 
   
   *Generate_Pads (sample_size_along_x_axis, sample_size_along_y_axis, [d], [d1], [d2], [layer], [contact pad coordinates])*. 
   
   
   *   **Transmission line (TL)**. Connects two contact pads from left to right. Generally, in this code everything that involves coplanar waveguides should be drawn from left to right.
   
   *Generate_TL (contact_pad_1, contact_pad2, d, d1, d2)*
   
    
   *   **Resonators** The position is specified by the left lower corner in (x,y) format. 
   
   
   *   **Ground grid**. Can be generated only after everything else is in place because we need to know in which areas no grid can be drawn (the 'restricted areas').
   
   *Generate_Grid* (sample_size_along_x, sample_size_along_y, layer1, layer2)
   
 If you want to change anything, it's most convenient to do a full restart (Restart & Run all).

In [1]:
!pip install gdspy

In [2]:
from designTL import * #this file contains the code to draw the pads and the transmission line
from Resonators_Purcell import* #this file contains the code to draw the resonators with coupled Purcell filters
import gdspy

In [3]:
class Sample:
    
    #Содержит функции, генерирующие все площадки, TL, все джозефсоны, все петельки
    
    def __init__(self, number):
        self.name = 'sample' + str(number)
        self.cell = gdspy.Cell(self.name)
    
    def Generate_Pads(self, a, b, number, d_arr, d1_arr, d2_arr, layer, coords = []):
        self.Pads = Pads(a, b, number, d_arr, d1_arr, d2_arr, layer, coords)
        self.Pads_cell = gdspy.Cell('Pads')
        self.Pads_cell.add(self.Pads.Generate_Ground())
        self.cell.add(gdspy.CellReference(self.Pads_cell, (0, 0)))
        
    def Generate_TL(self, start_pad, finish_pad, d, d1, d2, layer):
        TL = Transmission_Line(self.Pads, start_pad, finish_pad, d, d1, d2, layer)
        self.tl = TL
        self.tl_cell1 = gdspy.Cell('TL1')
        self.tl_cell2 = gdspy.Cell('TL2')
        self.tl_cell3 = gdspy.Cell('TL3')
        self.tl_cell1.add(self.tl.GenerateTL1(coords = [], rots = [])) 
        self.tl_cell2.add(TL.GenerateTL2(coords = [], rots = [])) 
        self.tl_cell3.add(TL.GenerateTL3(coords = [], rots = [])) 
        #self.cell.add([gdspy.CellReference(self.tl_cell, (0, 0))])
      
    def Generate_Resonators(self, start_points, freqs, modes, layer, ws, ss, TL_ground, ground, coupl_lengths, l_verts, freqsP):
        self.RESONATORS1 = gdspy.Cell('RESONATORS1')
        self.RESONATORS2 = gdspy.Cell('RESONATORS2')
        self.RESONATORS3 = gdspy.Cell('RESONATORS3')
        self.resonators = []
        self.res_cells = []
        self.res_references = []
        for i, (start, freq, mode, w, s, TL, ground, coupl_length, l_vert, freqP) in enumerate(zip(start_points, freqs, modes, ws, ss, TL_ground, ground, coupl_lengths, l_verts, freqsP)):
            self.resonators.append(Resonator(start.x, start.y, freq, layer, w, s, TL, ground, coupl_length, l_vert, freqP))
            self.res_cells.append(gdspy.Cell('res' + str(i)))
            self.RESONATORS1.add(self.resonators[i].Generate1(mode))
            self.RESONATORS2.add(self.resonators[i].Generate2(mode))
            self.RESONATORS3.add(self.resonators[i].Generate3(mode))
            self.res_cells[i].add(self.resonators[i].Generate3(mode))
            
        COMBINATION = gdspy.Cell('COMBINATION') #this cell will contain all the biggest polygons
        COMBINATION.add(gdspy.boolean(gdspy.CellReference(self.RESONATORS3, (0,0)), gdspy.CellReference(self.tl_cell3, (0,0)), 'or'))
        CUT = gdspy.Cell('CUT') #this cell will contain all the polygons that need to be cut out from the bigger ones
        CUT.add(gdspy.boolean(gdspy.CellReference(self.RESONATORS2, (0,0)), gdspy.CellReference(self.tl_cell2, (0,0)), 'or'))
        CUTOUT = gdspy.Cell('CUTOUT') #this cell will contain the result of the substraction of the previous two cells
        CUTOUT.add(gdspy.boolean(gdspy.CellReference(COMBINATION, (0,0)), gdspy.CellReference(CUT, (0,0)), 'not'))
        FILL = gdspy.Cell('FILL') #this cell will contain all the polygons that need to be placed in the void created by the previous cut
        FILL.add(gdspy.boolean(gdspy.CellReference(self.RESONATORS1, (0,0)), gdspy.CellReference(self.tl_cell1, (0,0)), 'or'))
        
        CUTOUTFILL = gdspy.Cell('CUTOUTFILL') #in this cell the filling and the cutout are combined.
        CUTOUTFILL.add(gdspy.boolean(gdspy.CellReference(CUTOUT, (0,0)), gdspy.CellReference(FILL, (0,0)), 'or'))
        self.cell.add(gdspy.CellReference(CUTOUTFILL, (0, 0))) #the final result is added to the main cell

                        
    def Generate_Grid(self, a, b, l1, l2):
        self.grid_cell = gdspy.Cell('Grid')
        res1 = gdspy.Rectangle((400, 400), (402, b - 400), layer = l1) 
        x = 447
        while x < a - 400:
            r1 = gdspy.Rectangle((x, 400), (x + 2, b - 400), layer = l1) 
            x += 47
            res1 = gdspy.boolean(res1, r1, 'or')
            #self.grid_cell.add(r1)
        y = 400
        res2 = gdspy.Rectangle((400, 400), (400, 402), layer = l2) 
        while y < b - 400:
            r1 = gdspy.Rectangle((400, y), (a - 400, y + 2), layer = l2) 
            y += 47
            res2 = gdspy.boolean(res2, r1, 'or')
            #self.grid_cell.add(r1)
        res2 = gdspy.boolean(res2, res1, 'not')
        for area in self.Pads.restricted_area:
            res1 = gdspy.boolean(res1, area, 'not', layer = l1)
            res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        for area in self.tl.restricted_area:
            res1 = gdspy.boolean(res1, area, 'not', layer = l1)
            res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        for i in range(len(self.resonators)):
            area = self.resonators[i].restricted_area
            res1 = gdspy.boolean(res1, area, 'not', layer = l1)
            res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        
        #res = gdspy.boolean(res, self.Pads_cell.get_polygonsets(), 'not')
        self.grid_h = res1
        self.grid_v = res2
        self.grid_cell.add(res1)
        self.grid_cell.add(res2)
        self.cell.add(gdspy.CellReference(self.grid_cell, (0, 0)))

In [4]:
sample = Sample(2)
TL_inner = 7 #the width of the inner conductor of the TL
TL_vacuum = 3 #the width of the 'vacuum' in between the inner and outer conductors of the Tl
TL_ground = 16 #the width of the outer conductors of the TL
sample.Generate_Pads(5000, 5000, #length and height of sample
                     2, #number of Pads
                     [TL_inner, TL_inner], #d
                     [TL_inner +2*TL_vacuum, TL_inner +2*TL_vacuum], #d1
                     [TL_inner + 2*(TL_vacuum+TL_ground), TL_inner + 2*(TL_vacuum+TL_ground)], #d2
                     0, #layer
                     [coordinates(800, 2500), coordinates(4200, 2500)]) #reference coordinates for Pads
sample.Generate_TL(0, 1, TL_inner, TL_inner +2*TL_vacuum, TL_inner + 2*(TL_vacuum+TL_ground), 0)
sample.Generate_Resonators([coordinates(sample.tl.start.x + 500, 
                                        sample.tl.start.y + (sample.tl.d2 - sample.tl.d)/2 + sample.tl.d),
                           coordinates(sample.tl.start.x + 500, 
                                        sample.tl.start.y + (sample.tl.d2 - sample.tl.d)/2 + sample.tl.d),
                           coordinates(sample.tl.start.x + 2000, 
                                        sample.tl.start.y + (sample.tl.d2 - sample.tl.d)/2 + sample.tl.d)], # ref coordinates
                           [6.2e9, 7e9, 6.4e9], #frequency of the bottom resonator 
                           ['up', 'down','up'], #whether the resonator is above or below the TL
                           [0, 0, 0], #layers
                           [4, 4, 4], #w the width of the inner resonator conductor
                           [4, 4, 4], #s, the distance between the inner and outer resonator conductors
                           [TL_ground, TL_ground, TL_ground],
                           ['noground', 'noground', 'noground'], #whether there is a ground (=conductor) between the TL and the resonator core
                           [1000, 1000, 1000], #coupling length between bottom resonator and TL
                           [500+1000, 500+1000, 2000+1000], #length of open end section
                           [6.2e9, 7e9, 6.4e9]) #Frequency of the top resonator

total resonator length: 4787.6769632524565
coupling length: 1000
w= 4
s= 4.0
w/s= 1.0
w+2s= 12
Purcell coupling length: 252
...
total resonator length: 4240.51388173789
coupling length: 1000
w= 4
s= 4.0
w/s= 1.0
w+2s= 12
Purcell coupling length: 252
...
total resonator length: 4638.062058150817
coupling length: 1000
w= 4
s= 4.0
w/s= 1.0
w+2s= 12
Purcell coupling length: 252
...


In [ ]:
gdspy.LayoutViewer(depth = 2) #this opens a viewer

In [ ]:
gdspy.write_gds('Resonator_Purcell.gds', name='library', unit=1e-06, precision=1e-09, timestamp=None, binary_cells=None)

In [ ]:
sample.Generate_Grid(10000,5000,0,0) #this draws the grid